In [1]:
from tqdm.notebook import tqdm
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F


import csv
import os
import sys
import time
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
Q = 3
M = 5
WINDOW_SIZE=1024

In [3]:
"""
Code here is copy and pasted from Stephen (@whistle_posse)
https://twitter.com/whistle_posse/status/1488656595114663939?s=20&t=lB_T74PcwZmlJ1rrdu8tfQ
from this notebook
https://github.com/StephenHogg/SCS/blob/main/SCS/layer.py
"""
class AbsPool(nn.Module):
    def __init__(self, pooling_module=None, *args, **kwargs):
        super(AbsPool, self).__init__()
        self.pooling_layer = pooling_module(*args, **kwargs)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pos_pool = self.pooling_layer(x)
        neg_pool = self.pooling_layer(-x)
        abs_pool = torch.where(pos_pool >= neg_pool, pos_pool, -neg_pool)
        return abs_pool


MaxAbsPool1d = partial(AbsPool, nn.MaxPool1d)
MaxAbsPool2d = partial(AbsPool, nn.MaxPool2d)
MaxAbsPool3d = partial(AbsPool, nn.MaxPool3d)

In [5]:
"""
Based on  and copy/pasted heavily from code
https://github.com/ZeWang95/scs_pytorch/blob/main/scs.py
from Ze Wang
https://twitter.com/ZeWang46564905/status/1488371679936057348?s=20&t=lB_T74PcwZmlJ1rrdu8tfQ

and code
https://github.com/oliver-batchelor/scs_cifar/blob/main/src/scs.py
from Oliver Batchelor
https://twitter.com/oliver_batch/status/1488695910875820037?s=20&t=QOnrCRpXpOuC0XHApi6Z7A

and the TensorFlow implementation
https://colab.research.google.com/drive/1Lo-P_lMbw3t2RTwpzy1p8h0uKjkCx-RB
and blog post
https://www.rpisoni.dev/posts/cossim-convolution/
from Raphael Pisoni
https://twitter.com/ml_4rtemi5
"""
import torch
from torch import nn
import torch.nn.functional as F


class SharpenedCosineSimilarity(nn.Module):
    def __init__(
        self,
        in_channels=1,
        out_channels=1,
        kernel_size=1,
        stride=1,
        padding=0,
        eps=1e-12,
    ):
        super(SharpenedCosineSimilarity, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.eps = eps
        self.padding = int(padding)

        w = torch.empty(out_channels, in_channels, kernel_size, kernel_size)
        nn.init.xavier_uniform_(w)
        self.w = nn.Parameter(
            w.view(out_channels, in_channels, -1), requires_grad=True)

        self.p_scale = 10
        p_init = 2**.5 * self.p_scale
        self.p = nn.Parameter(torch.empty(out_channels))
        nn.init.constant_(self.p, p_init)

        self.q_scale = 100
        self.q = nn.Parameter(torch.empty(1))
        nn.init.constant_(self.q, 10)

    def forward(self, x):
        # reshaping for compatibility with the einsum-based implementation
        w = self.w.reshape(
            self.out_channels,
            self.in_channels,
            self.kernel_size,
            self.kernel_size,
        )
        w_norm = torch.linalg.vector_norm(
            w,
            dim=(1, 2, 3),
            keepdim=True,
        )

        q_sqr = (self.q / self.q_scale) ** 2

        # a small difference: we add eps outside of the norm
        # instead of inside in order to reuse the performant
        # code of torch.linalg.vector_norm
        w_normed = w / ((w_norm + self.eps) + q_sqr)

        x_norm_squared = F.avg_pool1d(
            x ** 2,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
            #divisor_override=1, # we actually want sum_pool
        ).sum(dim=1, keepdim=True)

        y_denorm = F.conv1d(
            x,
            w_normed,
            bias=None,
            stride=self.stride,
            padding=self.padding,
        )

        y = y_denorm / ((x_norm_squared + self.eps).sqrt() + q_sqr)

        sign = torch.sign(y)

        y = torch.abs(y) + self.eps
        p_sqr = (self.p / self.p_scale) ** 2
        y = y.pow(p_sqr.reshape(1, -1, 1, 1))
        return sign * y


In [4]:

batch_size = 1024
n_epochs = 100
max_lr = .01
n_runs = 1000

# Allow for a version to be provided at the command line, as in
# $ python3 demo_fashion_mnist.py v15
if len(sys.argv) > 1:
    version = sys.argv[1]
else:
    version = "test"

training_set = CIFAR10(
    root=os.path.join('.', 'data', 'CIFAR10'),
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]))
testing_set = CIFAR10(
    root=os.path.join('.', 'data', 'CIFAR10'),
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]))

training_loader = DataLoader(
    training_set,
    batch_size=batch_size,
    shuffle=True,
    ) # num_workers=4)
testing_loader = DataLoader(
    testing_set,
    batch_size=batch_size,
    shuffle=False,
    ) # num_workers=4)


class Network(nn.Module):
    def __init__(self, window_size=1024, scs_kernel_size=16, abs_kernel_size=8):
        super().__init__()
        self.scs1 = SharpenedCosineSimilarity(
            in_channels=window_size, out_channels=window_size, kernel_size=scs_kernel_size, padding=0)
        self.pool1 = MaxAbsPool1d(kernel_size=abs_kernel_size, stride=2, ceil_mode=True)
        self.scs2 = SharpenedCosineSimilarity(
            in_channels=window_size, out_channels=window_size*2, kernel_size=scs_kernel_size, padding=0)
        self.pool2 = MaxAbsPool1d(kernel_size=abs_kernel_size, stride=2, ceil_mode=True)
        self.scs3 = SharpenedCosineSimilarity(
            in_channels=window_size*2, out_channels=window_size*4, kernel_size=scs_kernel_size, padding=0)
        self.pool3 = MaxAbsPool1d(kernel_size=abs_kernel_size, stride=2, ceil_mode=True)
        self.out = nn.Linear(in_features=window_size*4, out_features=window_size)

    def forward(self, t):
        t = self.scs1(t)
        t = self.pool1(t)

        t = self.scs2(t)
        t = self.pool2(t)

        t = self.scs3(t)
        t = self.pool3(t)

        t = t.reshape(-1, window_size*4)
        t = self.out(t)

        return torch.sigmoid(t)


# Restore any previously generated results.
try:
    accuracy_results = np.load(accuracy_results_path).tolist()
    accuracy_histories = np.load(accuracy_history_path).tolist()
    loss_results = np.load(loss_results_path).tolist()
except Exception:
    loss_results = []
    accuracy_results = []
    accuracy_histories = []

NameError: name 'CIFAR10' is not defined

In [6]:
class DataGenerator:
    """
    to inject anomalous points according to the formula in the paper:
    """
    def __init__(self, win_siz, step, nums):
        self.control = 0
        self.win_siz = win_siz
        self.step = step
        self.number = nums

    def generate_train_data(self, value, back_k=0, insert_anomaly=True):
        def normalize(a):
            amin = np.min(a)
            amax = np.max(a)
            a = (a - amin) / (amax - amin + 1e-5)
            return 3 * a

        if back_k <= 5:
            back = back_k
        else:
            back = 5
        length = len(value)
        tmp = []
        for pt in range(self.win_siz, length - back, self.step):
            head = max(0, pt - self.win_siz)
            tail = min(length - back, pt)
            data = np.array(value[head:tail])
            data = data.astype(np.float64)
            data = normalize(data)
            num = np.random.randint(1, self.number)
            ids = np.random.choice(self.win_siz, num, replace=False)
            lbs = np.zeros(self.win_siz, dtype=np.int64)
            if insert_anomaly:
                if (self.win_siz - 6) not in ids:
                    self.control += np.random.random()
                else:
                    self.control = 0
                if self.control > 100:
                    ids[0] = self.win_siz - 6
                    self.control = 0
                mean = np.mean(data)
                dataavg = average_filter(data)
                var = np.var(data)
                for id in ids:
                    data[id] += (dataavg[id] + mean) * np.random.randn() * min((1 + var), 10)
                    lbs[id] = 1
            tmp.append([data.tolist(), lbs.tolist()])
        return tmp

    
def average_filter(values, n=3):
    """
    Calculate the sliding window average for the give time series.
    Mathematically, res[i] = sum_{j=i-t+1}^{i} values[j] / t, where t = min(n, i+1)
    :param values: list.
        a list of float numbers
    :param n: int, default 3.
        window size.
    :return res: list.
        a list of value after the average_filter process.
    """

    if n >= len(values):
        n = len(values)

    res = np.cumsum(values, dtype=float)
    res[n:] = res[n:] - res[:-n]
    res[n:] = res[n:] / n

    for i in range(1, n):
        res[i] /= (i + 1)

    return res


def predict_next(values):
    """
    Predicts the next value by sum up the slope of the last value with previous values.
    Mathematically, g = 1/m * sum_{i=1}^{m} g(x_n, x_{n-i}), x_{n+1} = x_{n-m+1} + g * m,
    where g(x_i,x_j) = (x_i - x_j) / (i - j)
    :param values: list.
        a list of float numbers.
    :return : float.
        the predicted next value.
    """

    if len(values) <= 1:
        raise ValueError(f'data should contain at least 2 numbers')

    v_last = values[-1]
    n = len(values)

    slopes = [(v_last - v) / (n - 1 - i) for i, v in enumerate(values[:-1])]

    return values[1] + sum(slopes)


def extend_series(values, extend_num=M, look_ahead=M):
    """
    extend the array data by the predicted next value
    :param values: list.
        a list of float numbers.
    :param extend_num: int, default 5.
        number of values added to the back of data.
    :param look_ahead: int, default 5.
        number of previous values used in prediction.
    :return: list.
        The result array.
    """

    if look_ahead < 1:
        raise ValueError('look_ahead must be at least 1')

    extension = [predict_next(values[-look_ahead - 2:-1])] * extend_num
    return np.concatenate((values, extension), axis=0)

def load_kpi(csv_path):
    kpis = {}
    anomalies = 0
    with open(csv_path) as f:
        input = csv.reader(f, delimiter=',')
        cnt = 0
        for row in input:
            if cnt == 0:
                cnt += 1
                continue
            kpi = kpis.get(str(row[3]), [[], [], []])
            kpi[0].append(int(row[0]))  # timestamp
            kpi[1].append(float(row[1]))  # value
            kpi[2].append(int(row[2]))  # label
            kpis[str(row[3])] = kpi
            cnt += 1
            if int(row[2]) == 1:
                anomalies += 1
        print("Training data loaded. Total length: {}, number of anomalies: {}".format(cnt, anomalies))
        f.close()
    return kpis

In [7]:
kpis = load_kpi(os.getcwd() + '/../spectral-residual/train.csv')

Training data loaded. Total length: 3004067, number of anomalies: 79554


In [ ]:
training_data = []
generator = DataGenerator(WINDOW_SIZE, 1, 3)
for kpi in kpis.values():
    in_value = kpi[1]
    train_data = generator.generate_train_data(in_value)
    training_data += train_data

In [7]:
class SRCNN(nn.Module):
    def __init__(self, window=1024):
        self.window = window
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv1d(window, window, kernel_size=1, stride=1, padding=0)
        self.layer2 = nn.Conv1d(window, 2 * window, kernel_size=1, stride=1, padding=0)
        self.fc1 = nn.Linear(2 * window, 4 * window)
        self.fc2 = nn.Linear(4 * window, window)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = x.view(x.size(0), self.window, 1)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [8]:
model = SRCNN()

'/Users/i506171/IdeaProjects/upscale-sre-aiops/anomalydetection/notebooks'

In [15]:
torch.save(model.state_dict(), '/Users/i506171/IdeaProjects/upscale-sre-aiops/anomalydetection/serving/model.pth')

In [3]:
js=json.loads({"name": "startup.cfg", "modelCount": 1, "models": {\
  "testsrcnn": {\
    "1.0": {\
        "defaultVersion": "true",\
        "marName": "testsrcnn.mar",\
        "minWorkers": 1,\
        "maxWorkers": 1,\
        "batchSize": 8,\
        "maxBatchDelay": 50,\
        "responseTimeout": 120\
    }\
  }\
}}